In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import itertools
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import recall_score,precision_score,f1_score               

import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
import scipy
from keras import backend as K
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Dropout,Input,Embedding,RepeatVector,Permute,merge,Lambda
from keras.layers import multiply ,concatenate ,subtract,add,Add,Reshape                   
from keras.optimizers import RMSprop ,Adam
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model

In [22]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/train.csv')
train_feat = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/user_features.csv')

In [3]:
test_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/test.csv')

Checking if all ids in test are present in train_feat

In [4]:
train_data_new = train_data

In [5]:
train_data1 = train_data_new[['node1_id']]
train_data2 = train_data_new[['node2_id']]

y_train = train_data_new['is_chat']

In [6]:
test_data1 = test_data[['node1_id']]
test_data2 = test_data[['node2_id']]


In [39]:
train_data.shape

(70661802, 3)

In [118]:
train_data_new =train_data[0:7066180]

Bringing the Train Features

In [119]:
train_feat1 =  pd.merge(left = train_data_new, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [120]:
train_feat2 = pd.merge(left = train_data_new, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Bringing Test Features

In [70]:
test_feat1 = pd.merge(left = test_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [71]:
test_feat2 = pd.merge(left = test_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Standardizing Columns

In [81]:
cols_std = ['f1','f2','f3','f4','f5', 'f6','f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']

In [121]:
train_feat1 = train_feat1[cols_std]
train_feat2 = train_feat2[cols_std]


In [74]:
test_feat1 = test_feat1[cols_std]
test_feat2 = test_feat2[cols_std]

In [122]:
y_train = train_data_new['is_chat']

Baseline Random Forest


In [84]:
from sklearn.ensemble import RandomForestClassifier 

In [123]:
x_train = pd.concat([train_feat1, train_feat2], axis=1)

In [86]:
x_test = pd.concat([test_feat1, test_feat2], axis=1)

In [124]:
from sklearn.model_selection import train_test_split
x1_train, x1_val, y1_train, y1_val = train_test_split(x_train, y_train,test_size=0.2,random_state=0)

In [125]:
rf =  RandomForestClassifier(bootstrap=True, criterion='gini',
                max_depth=20, max_features='auto', max_leaf_nodes=None, 
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=6,
                oob_score=False, random_state=12, warm_start=False,
                class_weight={0:5,1:100}, verbose=1)

In [126]:
rf.fit(x1_train, y1_train)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  5.4min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed: 13.3min finished


RandomForestClassifier(bootstrap=True, class_weight={0: 5, 1: 100},
            criterion='gini', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=6, oob_score=False, random_state=12,
            verbose=1, warm_start=False)

In [129]:
print (rf.score(x1_val, y1_val))


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.0s


0.9215332754048157


[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:   14.3s finished


In [127]:
print (recall_score(y1_val, rf.predict(x1_val)))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    5.5s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:   12.6s finished


0.4428558663211509


In [128]:
print (precision_score(y1_val, rf.predict(x1_val)))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:   14.4s finished


0.18741491453637876


In [ ]:
#y1_train[y1_train==0].shape

In [130]:
rf_pred = rf.predict_proba(x_test)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   55.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  2.2min finished


In [137]:
#rf_pred[0:100]

In [132]:
y_ones = rf_pred[:,1]

In [133]:
sub_df = test_data.copy()

In [138]:
sub_df['is_chat'] = y_ones

In [139]:
sub_df = sub_df[['id','is_chat']]

In [140]:
z = np.load('/Users/s0c02nj/Desktop/Hike/same_node_ids.npy')

In [145]:
#sub_df

In [147]:
z_set = set(z)

In [150]:
out = []
for i in tqdm(range(0,len(sub_df))):
    
    i_id = sub_df['id'].iloc[i]
    
    if i_id in z_set:
        k=0
    else:
        k = sub_df['is_chat'].iloc[i]
        
    out.append(k)

100%|██████████| 11776968/11776968 [05:23<00:00, 36366.62it/s]


In [152]:
sub_df['is_chat'] = out

In [153]:
sub_df.to_csv('/Users/s0c02nj/Desktop/Hike/Submission.csv',index=False)

In [ ]:
#train_feat1.shape
#y_cat.shape

In [ ]:
def model_deep():
    
    #Defining the input-----> Continuous Variable
    inputs1 = Input(shape=(14,))
    
    #Defining the input-----> Categorical Variable
    inputs2 = Input(shape=(14,))
    
    #Defining the -----> Complaint-reason
    layer_merge = concatenate([inputs1,inputs2])
    
    #Dense Layer-----> This will be the Weight Matrix W (n*k)
    layer_dense = Dense(50, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    layer_dense = Dense(30, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    layer_dense = Dense(20, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    
    #Total Loss to Minimize
    loss_total = Dense(2,activation='softmax')(layer_dense)
    
    
    model = Model(inputs=[inputs1,inputs2], outputs= loss_total)
    return model


In [ ]:
model = model_deep()
model.summary()

In [ ]:
import tensorflow as tf
from keras import backend as K

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.001), metrics = [auc])

In [ ]:
#Deep Learning Model
history = model.fit([train_feat1,
                     train_feat2] ,
                     y_cat, batch_size = 1024, epochs = 3, validation_split=0.1,
                     verbose = 1) #,class_weight = {0:0.25 ,1:0.01 ,2:0.25 ,3:0.24 ,4:0.25} 



In [ ]:
#Tets
y_prob = model.predict([test_feat1,test_feat2])


In [ ]:
y_prob

In [ ]:
y_ones = y_prob[:,1]


In [ ]:
#Pred
y_classes = y_prob.argmax(axis=-1)

In [ ]:
sub_df = test_data.copy()

In [ ]:
sub_df['is_chat'] = y_ones

In [ ]:
sub_df = sub_df[['id','is_chat']]

In [ ]:
sub_df.to_csv('/Users/s0c02nj/Desktop/Hike/Submission3.csv',index=False)

In [ ]:
#sub_df.shape

In [ ]:
#test_data.shape

In [ ]:
10490000
70661802

In [ ]:
train_data.shape